# Importing the datasets 


In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer

true = pd.read_csv('../raw_data/True.csv', nrows = 1000)
fake = pd.read_csv('../raw_data/Fake.csv', nrows = 1000)

In [2]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


# Dropping unseful features - Move to a binary classification problem

In [4]:
true.drop(columns = ['date', 'subject'], inplace = True)
fake.drop(columns = ['date', 'subject'], inplace = True)

In [14]:
true['score']=1
fake['score']=0

# Concatenating the datasets

In [15]:
data = pd.concat([true,fake], ignore_index=True)


In [16]:
stop_words =['/Getty Images']
pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
data['text'] = data['text'].str.replace(pat, '')


# Basic preprocessing

In [21]:
import string
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
data['text'] = data['text'].map(remove_punctuation)
data['title'] = data['title'].map(remove_punctuation)
def lower_case(text):
    text = text.lower()
    return text
data['text'] = data['text'].map(lower_case)
data['title'] = data['title'].map(lower_case)
def remove_numbers(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text
data['text'] = data['text'].map(remove_numbers)
data['title'] = data['title'].map(remove_numbers)













In [24]:
data.drop(columns = ['Score'], inplace = True)

In [25]:
data.head()

,title,text,score
0,as us budget fight looms republicans flip thei...,washington reuters the head of a conservative...,1
1,us military to accept transgender recruits on ...,washington reuters transgender people will be...,1
2,senior us republican senator let mr mueller do...,washington reuters the special counsel invest...,1
3,fbi russia probe helped by australian diplomat...,washington reuters trump campaign adviser geo...,1
4,trump wants postal service to charge much more...,seattlewashington reuters president donald tr...,1


In [43]:
x = data['text']
y = data['score']

#y.reshape((2000,1))


# Splitting

In [44]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3)

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
# Create Pipeline
pipeline = Pipeline([
    ('cvt', CountVectorizer()),
    ('nb', MultinomialNB()),
])
# Set parameters to search
parameters = {
    'cvt__ngram_range': ((1,1), (2,2),(3,3),(4,4)),
    'nb__alpha': (0.1,1,2,5,10),}
# Perform grid search
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,
                           scoring = "accuracy",
                           refit=True, cv=5, verbose = 1)
grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   54.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvt', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'cvt__ngram_range': ((1, 1), (2, 2), (3, 3), (4, 4)), 'nb__alpha': (0.1, 1, 2, 5, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [46]:
grid_search.best_params_

{'cvt__ngram_range': (2, 2), 'nb__alpha': 10}

In [47]:
grid_search.best_score_

0.9842857142857143

In [50]:
grid_search.predict(x_test)

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,

In [52]:
x_test

405     washington reuters  two days after a federal c...
1190    if you haven t noticed no one associated with ...
1132    history has shown that one of the most costeff...
731     beijing reuters  us secretary of state rex til...
1754    when donald trump kicked off  made in america ...
1178    special counsel robert mueller appears to have...
1533    this year america has seen an interesting and ...
1303    president donald trump has had some strong wor...
1857    ronald reagan is rolling in his gravedonald tr...
18      reuters  a us appeals court in washington on t...
1266    in the wake of any tragedy in this country the...
1543    in the wake of donald trump s unhinged and ram...
249     washington reuters  republican us representati...
191     reuters  democrat doug jones won alabama’s ele...
721     beijingshanghai reuters  president donald trum...
1896    get ready for donald trump to throw the worst ...
452     london reuters  britain’s ambassador to the un...
1947    sunday

In [59]:
grid_search.score(x_test, y_test)

0.9883333333333333

In [62]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range= (2, 2))
x = vectorizer.fit_transform(x)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3)

In [63]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha = 10)
model.fit(x_train,y_train)


model.score(x_test,y_test)

0.9883333333333333